<center>

**Vježba 4 - PyTorch i MLflow**

</center>

U ovoj vježbi želimo pokazati naprednije korištenje *MLflow* servera na CIFAR10 skupu podataka. Za to će nam biti potreban PyTorch (CPU) python modul.  

Prvo učitavamo potrebne python module.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

import mlflow
import mlflow.pytorch
from mlflow.models.signature import infer_signature
from mlflow.client import MlflowClient
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository

Prvo učitavamo CIFAR10 podskupove za učenje i testiranje korištenjem *torchvision* python modula.

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='~/torch_files/', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='~/torch_files/', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Mala lokalna demonstracija par slika iz CIFAR10 modula.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(images):
    img = torchvision.utils.make_grid(images)
    img = img / 2 + 0.5 # denormaliziranje
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    return plt


dataiter = iter(trainloader)
images, labels = next(dataiter)

# par slučajnih slika
plt = imshow(images)
plt.show()
# labele
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

Definiramo CNN za CIFAR10 skup podataka. Dimenzije tensora su podešene ručno.

In [ ]:
class ConvCifar10Net(nn.Module):
    def __init__(self):
        super(ConvCifar10Net,self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Postavimo *MLflow* server URL. Zatim potražimo naš CIFAR10 eksperiment.  
U slučaju da eksperiment nije pronađen, stvorimo ga.

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")
exps = mlflow.search_experiments(filter_string="name LIKE '%CIFAR10%'")
if len(exps)==0:
    exp_id = mlflow.create_experiment("Deep CNN - CIFAR10 dataset")
else:
    exp_id = exps[0].experiment_id

Prvo naučimo CNN na podskupu za učenje.  
U procesu učenja detektiramo sučelje CNN-a na temelju podskupa za učenje - to će biti PyTorch tensori.  
 
U našem pokretanju zapišemo PyTorch model koji smo prethodno naučili.   
Posebno definiramo meta-podatke o modelu koji smo upisali.  
Obratite pažnju na ```log_metric``` pozive koji se dešavaju u toku učenja modela.

In [ ]:

with mlflow.start_run(run_name="Training CIFAR10 CNN",
                        experiment_id=exp_id) as run:
    model = ConvCifar10Net()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.5)
    mlflow.log_param("learning_rate", 0.001)
    mlflow.log_param("momentum", 0.5)

    epochs = 10
    mlflow.log_param("epochs", epochs)

    dataiter = iter(trainloader)
    images, labels = next(dataiter)
    plt = imshow(images)
    mlflow.log_figure(plt.gcf(), "examples.png")
    plt.close()

    schema = None

    for epoch in range(epochs):
        running_loss,met_loss = 0.0,0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data

            optimizer.zero_grad()

            outputs = model(inputs)
            if schema is None:
                schema = infer_signature(inputs.detach().numpy(), outputs.detach().numpy())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            met_loss += loss.item()
            if i % 50 == 49:
                mlflow.log_metric("loss", met_loss/50, (epoch*len(trainloader))+i)
                met_loss = 0.0
            if i % 2000 == 1999:  
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0


    mlflow.pytorch.log_model(model, artifact_path="pytorch-model", signature=schema)
        
    name="CIFAR10 CNN"

    client = MlflowClient()
    rms = client.search_registered_models("name='"+name+"'")
    if len(rms)==0: client.create_registered_model(name)

    model_uri = "runs:/{}/pytorch-model".format(run.info.run_id)
    model_src = RunsArtifactRepository.get_underlying_uri(model_uri)
    v = client.create_model_version(name, model_src, run.info.run_id, 
        description="CNN model for CIFAR10")

mlflow.end_run()

Radimo ručnu evaluaciju zbog ```torch.max``` "sloja" koji još uvijek nije podržana u *MLflow* serveru.  
Ručnu evaluaciju zapisujemo kao posebno pokretanje.

Posebno obratiti pozornost na ```log_figure``` koji nam omogućava spremanje custom evaluacije u obliku grafikona.

In [9]:
import pandas as pd

name="CIFAR10 CNN"
client = MlflowClient()
models = client.get_latest_versions(name, stages=["None"])

with mlflow.start_run(run_name="Manual test of CIFAR10 CNN, version:"+models[0].version,
                        experiment_id=exp_id) as run:
    model = mlflow.pytorch.load_model(models[0].source)

    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}
    conf_mat = [[0 for x in range(len(classes))] for y in range(len(classes))]

    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            for label, prediction in zip(labels, predictions):
                conf_mat[label][prediction] += 1
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1
    
    acc = []
    for classname in classes: acc.append((correct_pred[classname]/total_pred[classname])*100)
    fig = plt.figure(figsize=(7,7))
    plt.bar(list(classes), acc)
    plt.xlabel('Class', fontsize=18)
    plt.ylabel('Accuracy (%)', fontsize=18)
    mlflow.log_figure(fig, "classes_accuracy.png")
    plt.close()

    fig, ax = plt.subplots(figsize=(10,10))
    cl = list(classes)
    ax.matshow(conf_mat, cmap=plt.cm.Wistia, alpha=0.6)
    for i in range(len(cl)):
        for j in range(len(cl)):
            ax.text(x=j, y=i, s='{}'.format(conf_mat[i][j]), va='center', ha='center', size='large')
    
    plt.xticks(range(len(cl)), cl)
    plt.yticks(range(len(cl)), cl)

    plt.xlabel('Predictions', fontsize=18)
    plt.ylabel('Actuals', fontsize=18)
    plt.title('Confusion Matrix - CIFAR10', fontsize=18)
    mlflow.log_figure(fig,"confusion_matrix.png")
    plt.close()

mlflow.end_run()